In [35]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar bibliotecas
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

# Carregar o dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/heart_failure_clinical_records_dataset.csv'

data = pd.read_csv(file_path)
print(data.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.

In [36]:
# Preparar variáveis preditoras (X) e alvo (y)
X = data.drop('DEATH_EVENT', axis=1)
y = data['DEATH_EVENT']

# Dividir em treino e teste (70% treino, 30% teste), com estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

# Treinar os modelos
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

mlp = MLPClassifier(random_state=42, max_iter=500)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

# Função para calcular métricas
def calcular_metricas(y_true, y_pred):
    return {
        'Acurácia': accuracy_score(y_true, y_pred),
        'F1-Score': f1_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'Precisão': precision_score(y_true, y_pred)
    }

# Calcular métricas
metricas_dt = calcular_metricas(y_test, y_pred_dt)
metricas_svm = calcular_metricas(y_test, y_pred_svm)
metricas_mlp = calcular_metricas(y_test, y_pred_mlp)

# Criar DataFrame para exibir resultados
df_metricas = pd.DataFrame([metricas_dt, metricas_svm, metricas_mlp],
                           index=['Árvore de Decisão', 'SVM Linear', 'MLP Neural Network'])

print("Métricas dos Modelos:")
print(df_metricas.round(4))

# Imprimir matrizes de confusão
print('\nMatrizes de Confusão:')
print('Árvore de Decisão:')
print(confusion_matrix(y_test, y_pred_dt))
print('\nSVM Linear:')
print(confusion_matrix(y_test, y_pred_svm))
print('\nMLP Neural Network:')
print(confusion_matrix(y_test, y_pred_mlp))


Métricas dos Modelos:
                    Acurácia  F1-Score  Recall  Precisão
Árvore de Decisão     0.7889    0.6545  0.6207    0.6923
SVM Linear            0.8000    0.6250  0.5172    0.7895
MLP Neural Network    0.3333    0.4915  1.0000    0.3258

Matrizes de Confusão:
Árvore de Decisão:
[[53  8]
 [11 18]]

SVM Linear:
[[57  4]
 [14 15]]

MLP Neural Network:
[[ 1 60]
 [ 0 29]]


O modelo de Árvore de Decisão apresentou um desempenho equilibrado, com uma acurácia de 78,9% e um F1-Score razoável (0,65). Ele conseguiu identificar cerca de 62% dos casos positivos reais (recall) e manteve uma precisão moderada (69%), o que indica um bom equilíbrio entre identificar corretamente os positivos e evitar falsos positivos. A matriz de confusão mostra que o modelo cometeu poucos erros, tanto em falsos positivos quanto em falsos negativos.

O SVM Linear teve uma acurácia ligeiramente maior (80%), porém seu recall foi menor (51,7%), o que significa que deixou passar quase metade dos casos positivos. Por outro lado, sua precisão foi alta (79%), indicando que, quando o modelo previu um caso positivo, geralmente acertou. Isso sugere que o SVM é mais conservador, classificando menos exemplos como positivos para evitar falsos positivos, mas perdendo alguns positivos reais.

O modelo MLP Neural Network apresentou o pior desempenho geral, com uma acurácia baixa (33,3%) e baixa precisão (32,6%), apesar de alcançar um recall perfeito (100%). Isso indica que o MLP classificou praticamente todos os exemplos como positivos, o que garantiu identificar todos os positivos reais, porém resultou em muitos falsos positivos, conforme confirmado pela matriz de confusão que mostra 60 falsos positivos e apenas 1 verdadeiro negativo.